<a href="https://colab.research.google.com/github/antoniomuso/speech2face/blob/master/Decoder_with_warping_newDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ! rm -r Face_Feature

# ! ls -l Face_Feature/Faces | wc -l
! pip install face_recognition
! pip3 install tensorflow-gpu==1.15

In [3]:
! cp /content/drive/My\ Drive/Speech2Face/face_features_10_per_actor.zip .
! unzip face_features_10_per_actor.zip

Archive:  face_features_10_per_actor.zip
replace content/Face_Feature/Faces/John_Turturro_11427.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [4]:
# !wget http://www.robots.ox.ac.uk/~vgg/research/CMBiometrics/data/zippedFaces.tar.gz 
# tar zxvf zippedFaces.tar.gz -C "/content/drive/My Drive/Speech2Face/vox1_dataset/"

In [5]:
# !curl --user voxceleb1912:0s42xuw6 -o "/content/drive/My Drive/Speech2Face/ff/vox.zip" http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox1_test_wav.zip

In [6]:
import glob

import numpy as np
import sys

from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import  torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from face_recognition import face_landmarks, face_locations, face_encodings
from face_recognition import load_image_file
from random import randint

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import os
from os.path import join
from os import listdir

from tqdm import tqdm
from google.colab.patches import cv2_imshow
import cv2

In [7]:
# Reproducibility stuff

import random
torch.manual_seed(42)
np.random.seed(42)
random.seed(0)

In [31]:
# This dataset use .npy embedding VGG
def load_metadata(data_file_path):
    data_list = []
    with open(data_file_path, "r") as f:
        for line in f:
            data_list.append(line.rstrip("\n").split(","))
    return data_list


class _Normalize_Tensor(object):
    def __init__(self, color_mean, color_std):
        self.color_mean = color_mean
        self.color_std = color_std

    def __call__(self, img):
        # Convert image to Tensor
        img = transforms.functional.to_tensor(img)

        # Normalize image by the parameter of pre-trained face-encoder
        img = transforms.functional.normalize(
            img, self.color_mean, self.color_std)

        return img

color_mean, color_std = [129.186279296875, 104.76238250732422, 93.59396362304688], [1,1,1]
color_mean = [tmp / 255.0 for tmp in color_mean]
color_std = [tmp / 255.0 for tmp in color_std]

class EmbedImagePairs(Dataset):
    def __init__(self, data_list_path, size=64):
        super().__init__()
        self.face_features = np.load(join(data_list_path,'facefeature.npy'))
        self.image_path = join(data_list_path, 'Faces')
        self.images =  listdir(join(data_list_path, 'Faces'))
        
        self.size = size


        self.transform_fd = transforms.Compose([
            #transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])

        self.transform_fe = transforms.Compose([
            _Normalize_Tensor(color_mean, color_std)
        ])        
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # Load face-image
        image_p = self.images[idx]
        features_index = int(image_p.split('_')[-1].split('.')[0])

        image = Image.open(join(self.image_path,image_p))
        image = image.resize((224,224))

        #display(image)

        #tr = transforms.ToTensor()(image)
        #print("TR", tr.shape, tr.dtype)

        #np_image = self.transform_fd(image).numpy()
        #print("1",np_image.dtype, np_image.shape)
        
        # Localize Face
        np_image = np.array(image)
        #print("np",np_image.dtype, np_image.shape)

        #ts = torch.from_numpy(np_image)
        #print("TS", ts.dtype, ts.shape)

        norm = self.transform_fe(np_image)
        
        #import matplotlib.pyplot as plt
        #plt.imshow(norm.permute(1, 2, 0))

        #print("norm", norm.dtype, norm.shape)

        #tf = self.transform_fd(image)
        #print("TF", tf.dtype, tf.shape)

        # print(np_image.shape)
        try:

          resultant = []
          #np_image = np_image[x:x1,y:y1]
          landmark = face_landmarks(np_image)
          for i in list(landmark[0].keys()):
            resultant += landmark[0][i]
          landmark = np.ravel(np.array(resultant))

        except IndexError:
          return self[randint(0, len(self))]

        face_vectors = torch.tensor(self.face_features[features_index])

        return face_vectors.float(), torch.tensor(landmark).float(), norm

#input_path = '/content/content/Face_Feature/'
#print('Image Pairing')
#dataset = EmbedImagePairs(input_path)
#print(dataset[0])

In [9]:
class Vgg_face_dag(nn.Module):

    def __init__(self):
        super(Vgg_face_dag, self).__init__()
        self.meta = {'mean': [129.186279296875, 104.76238250732422, 93.59396362304688],
                     'std': [1, 1, 1],
                     'imageSize': [224, 224, 3]}
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.fc6 = nn.Linear(in_features=25088, out_features=4096, bias=True)
        self.relu6 = nn.ReLU(inplace=True)
        self.dropout6 = nn.Dropout(p=0.5)
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.relu7 = nn.ReLU(inplace=True)
        self.dropout7 = nn.Dropout(p=0.5)
        self.fc8 = nn.Linear(in_features=4096, out_features=2622, bias=True)

    def forward(self, x0, is_fc8=False):
        x1 = self.conv1_1(x0)
        x2 = self.relu1_1(x1)
        x3 = self.conv1_2(x2)
        x4 = self.relu1_2(x3)
        x5 = self.pool1(x4)
        x6 = self.conv2_1(x5)
        x7 = self.relu2_1(x6)
        x8 = self.conv2_2(x7)
        x9 = self.relu2_2(x8)
        x10 = self.pool2(x9)
        x11 = self.conv3_1(x10)
        x12 = self.relu3_1(x11)
        x13 = self.conv3_2(x12)
        x14 = self.relu3_2(x13)
        x15 = self.conv3_3(x14)
        x16 = self.relu3_3(x15)
        x17 = self.pool3(x16)
        x18 = self.conv4_1(x17)
        x19 = self.relu4_1(x18)
        x20 = self.conv4_2(x19)
        x21 = self.relu4_2(x20)
        x22 = self.conv4_3(x21)
        x23 = self.relu4_3(x22)
        x24 = self.pool4(x23)
        x25 = self.conv5_1(x24)
        x26 = self.relu5_1(x25)
        x27 = self.conv5_2(x26)
        x28 = self.relu5_2(x27)
        x29 = self.conv5_3(x28)
        x30 = self.relu5_3(x29)
        x31_preflatten = self.pool5(x30)
        x31 = x31_preflatten.view(x31_preflatten.size(0), -1)
        x32 = self.fc6(x31)
        x33 = self.relu6(x32)
        x34 = self.dropout6(x33)
        x35 = self.fc7(x34)
        x36 = self.relu7(x35)
        x37 = self.dropout7(x36)
        if is_fc8:
            x38 = self.fc8(x37)
        else:
            x38 = x37
        return x38


def vgg_face_dag(weights_path=None, **kwargs):
    """
    load imported model instance
    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = Vgg_face_dag()
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model

In [10]:
class DECODER(nn.Module):
    def __init__(self, phase):
        super(DECODER, self).__init__()
        self.phase = phase
        self.fc3 = nn.Linear(4096, 1000)
        self.ReLU = nn.ReLU()
        #self.fc_bn3 = nn.BatchNorm1d(1000)


        self.fc4 = nn.Linear(1000, 14 * 14 * 256)
        self.fc_bn4 = nn.BatchNorm1d(14 * 14 * 256)
        def TransConv( i, kernal = 5, stride = 2, inp = None):
            if not inp:
                inp = max(256//2**(i-1), 32)

            layer =  nn.Sequential(
                nn.ConvTranspose2d(inp, max(256//2**i, 32), 
                                kernal, stride=stride, padding=2, output_padding=1, 
                                dilation=1, padding_mode='zeros'),
                nn.ReLU(),
                nn.BatchNorm2d(max(256//2**i, 32)))
            return layer
        self.T1_ = TransConv(1, inp = 256)
        self.T2_ = TransConv(2)
        self.T3_ = TransConv(3)
        self.T4_ = TransConv(4)
    
        self.ConvLast = nn.Sequential(
            nn.Conv2d(32, 3, (1,1), stride=1),
            nn.BatchNorm2d(3),
            nn.ReLU())


        self.layerLandmark1 = nn.Linear(1000, 800)
        self.layerLandmark2 = nn.Linear(800, 600)
        self.layerLandmark3 = nn.Linear(600, 400)
        self.layerLandmark4 = nn.Linear(400, 200)
        self.layerLandmark5 = nn.Linear(200, 144)

    def forward(self, x):
        L1 = self.fc3(x)
        L1 = self.ReLU(L1)


        L2 = self.layerLandmark1(L1)
        L2 = self.ReLU(L2)

        L3 = self.layerLandmark2(L2)
        L3 = self.ReLU(L3)

        L4 = self.layerLandmark3(L3)
        L4 = self.ReLU(L4)

        L5 = self.layerLandmark4(L4)
        L5 = self.ReLU(L5)

        L6 = self.layerLandmark5(L5)
        outL = self.ReLU(L6)


        # B1 = self.fc_bn3(L1) 
        T0 = self.fc4(L1) 
        T0 = self.ReLU(T0)
        # T0 = self.fc_bn4(T0)
        T0 = T0.view(-1,256,14,14)



        T1 = self.T1_(T0)
        T2 = self.T2_(T1)
        T3 = self.T3_(T2)
        T4 = self.T4_(T3)

        outT = self.ConvLast(T4)
        if self.phase == "train":
            return outL,  outT 

In [11]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms
        
    def __call__(self, x):
        for t in self.transforms:
            x = t(x)
        return x
    
class Resize(object):
    def __init__(self, input_size):
        self.input_size = input_size
        
    def __call__(self, img):
        img = img.resize((self.input_size, self.input_size),
                                Image.BICUBIC)
        return img
    
class Normalize_Tensor(object):
    def __init__(self, color_mean, color_std):
        self.color_mean = color_mean
        self.color_std = color_std

    def __call__(self, img):

        # PIL画像をTensorに。大きさは最大1に規格化される
        img = transforms.functional.to_tensor(img)

        # 色情報の標準化
        img = transforms.functional.normalize(
            img, self.color_mean, self.color_std)

        return img

class ImageTransform():
    def __init__(self, input_size, color_mean, color_std):
        self.data_transform = {
            'train' : Compose([
                Resize(input_size),
                Normalize_Tensor(color_mean, color_std)
            ]),
            'val' : Compose([
                Resize(input_size),
                Normalize_Tensor(color_mean, color_std)
            ])
        }
    
    def __call_(self, phase, img):
        return self.data_transform[phase](img)

In [12]:
############### THIS CELL IS COPIED FROM DEEP SPEECH MOZILLA #########################


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import tensorflow.compat.v1 as tfv1
from tensorflow.compat import dimension_value
from tensorflow.contrib.image import dense_image_warp
from tensorflow.contrib.image import interpolate_spline

from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops

def _to_float32(value):
    return tf.cast(value, tf.float32)

def _to_int32(value):
    return tf.cast(value, tf.int32)

def _get_grid_locations(image_height, image_width):
    """Wrapper for np.meshgrid."""
    tfv1.assert_type(image_height, tf.int32)
    tfv1.assert_type(image_width, tf.int32)

    y_range = tf.range(image_height)
    x_range = tf.range(image_width)
    y_grid, x_grid = tf.meshgrid(y_range, x_range, indexing='ij')
    return tf.stack((y_grid, x_grid), -1)


def _expand_to_minibatch(tensor, batch_size):
    """Tile arbitrarily-sized np_array to include new batch dimension."""
    ndim = tf.size(tf.shape(tensor))
    ones = tf.ones((ndim,), tf.int32)

    tiles = tf.concat(([batch_size], ones), 0)
    return tf.tile(tf.expand_dims(tensor, 0), tiles)


def _get_boundary_locations(image_height, image_width, num_points_per_edge):
    """Compute evenly-spaced indices along edge of image."""
    image_height_end = _to_float32(tf.math.subtract(image_height, 1))
    image_width_end = _to_float32(tf.math.subtract(image_width, 1))
    y_range = tf.linspace(0.0, image_height_end, num_points_per_edge + 2)
    x_range = tf.linspace(0.0, image_height_end, num_points_per_edge + 2)
    ys, xs = tf.meshgrid(y_range, x_range, indexing='ij')
    is_boundary = tf.logical_or(
        tf.logical_or(tf.equal(xs, 0.0), tf.equal(xs, image_width_end)),
        tf.logical_or(tf.equal(ys, 0.0), tf.equal(ys, image_height_end)))
    return tf.stack([tf.boolean_mask(ys, is_boundary), tf.boolean_mask(xs, is_boundary)], axis=-1)


def _add_zero_flow_controls_at_boundary(control_point_locations,
                                        control_point_flows, image_height,
                                        image_width, boundary_points_per_edge):
    """Add control points for zero-flow boundary conditions.
     Augment the set of control points with extra points on the
     boundary of the image that have zero flow.
    Args:
      control_point_locations: input control points
      control_point_flows: their flows
      image_height: image height
      image_width: image width
      boundary_points_per_edge: number of points to add in the middle of each
                             edge (not including the corners).
                             The total number of points added is
                             4 + 4*(boundary_points_per_edge).
    Returns:
      merged_control_point_locations: augmented set of control point locations
      merged_control_point_flows: augmented set of control point flows
    """

    batch_size = dimension_value(tf.shape(control_point_locations)[0])

    boundary_point_locations = _get_boundary_locations(image_height, image_width,
                                                       boundary_points_per_edge)
    boundary_point_shape = tf.shape(boundary_point_locations)
    boundary_point_flows = tf.zeros([boundary_point_shape[0], 2])

    minbatch_locations = _expand_to_minibatch(boundary_point_locations, batch_size)
    type_to_use = control_point_locations.dtype
    boundary_point_locations = tf.cast(minbatch_locations, type_to_use)

    minbatch_flows = _expand_to_minibatch(boundary_point_flows, batch_size)

    boundary_point_flows = tf.cast(minbatch_flows, type_to_use)

    merged_control_point_locations = tf.concat(
        [control_point_locations, boundary_point_locations], 1)

    merged_control_point_flows = tf.concat(
        [control_point_flows, boundary_point_flows], 1)

    return merged_control_point_locations, merged_control_point_flows


def sparse_image_warp(image,
                      source_control_point_locations,
                      dest_control_point_locations,
                      interpolation_order=2,
                      regularization_weight=0.0,
                      num_boundary_points=0,
                      name='sparse_image_warp'):
    """Image warping using correspondences between sparse control points.
    Apply a non-linear warp to the image, where the warp is specified by
    the source and destination locations of a (potentially small) number of
    control points. First, we use a polyharmonic spline
    (`tf.contrib.image.interpolate_spline`) to interpolate the displacements
    between the corresponding control points to a dense flow field.
    Then, we warp the image using this dense flow field
    (`tf.contrib.image.dense_image_warp`).
    Let t index our control points. For regularization_weight=0, we have:
    warped_image[b, dest_control_point_locations[b, t, 0],
                    dest_control_point_locations[b, t, 1], :] =
    image[b, source_control_point_locations[b, t, 0],
             source_control_point_locations[b, t, 1], :].
    For regularization_weight > 0, this condition is met approximately, since
    regularized interpolation trades off smoothness of the interpolant vs.
    reconstruction of the interpolant at the control points.
    See `tf.contrib.image.interpolate_spline` for further documentation of the
    interpolation_order and regularization_weight arguments.
    Args:
      image: `[batch, height, width, channels]` float `Tensor`
      source_control_point_locations: `[batch, num_control_points, 2]` float
        `Tensor`
      dest_control_point_locations: `[batch, num_control_points, 2]` float
        `Tensor`
      interpolation_order: polynomial order used by the spline interpolation
      regularization_weight: weight on smoothness regularizer in interpolation
      num_boundary_points: How many zero-flow boundary points to include at
        each image edge.Usage:
          num_boundary_points=0: don't add zero-flow points
          num_boundary_points=1: 4 corners of the image
          num_boundary_points=2: 4 corners and one in the middle of each edge
            (8 points total)
          num_boundary_points=n: 4 corners and n-1 along each edge
      name: A name for the operation (optional).
      Note that image and offsets can be of type tf.half, tf.float32, or
      tf.float64, and do not necessarily have to be the same type.
    Returns:
      warped_image: `[batch, height, width, channels]` float `Tensor` with same
        type as input image.
      flow_field: `[batch, height, width, 2]` float `Tensor` containing the dense
        flow field produced by the interpolation.
    """

    image = ops.convert_to_tensor(image)
    source_control_point_locations = ops.convert_to_tensor(
        source_control_point_locations)
    dest_control_point_locations = ops.convert_to_tensor(
        dest_control_point_locations)

    control_point_flows = (
        dest_control_point_locations - source_control_point_locations)

    clamp_boundaries = num_boundary_points > 0
    boundary_points_per_edge = num_boundary_points - 1

    with ops.name_scope(name):
        image_shape = tf.shape(image)
        batch_size, image_height, image_width = image_shape[0], image_shape[1], image_shape[2]

        # This generates the dense locations where the interpolant
        # will be evaluated.
        grid_locations = _get_grid_locations(image_height, image_width)

        flattened_grid_locations = tf.reshape(grid_locations,
                                              [tf.multiply(image_height, image_width), 2])

        # flattened_grid_locations = constant_op.constant(
        #     _expand_to_minibatch(flattened_grid_locations, batch_size), image.dtype)
        flattened_grid_locations = _expand_to_minibatch(flattened_grid_locations, batch_size)
        flattened_grid_locations = tf.cast(flattened_grid_locations, dtype=image.dtype)

        if clamp_boundaries:
            (dest_control_point_locations,
             control_point_flows) = _add_zero_flow_controls_at_boundary(
                 dest_control_point_locations, control_point_flows, image_height,
                 image_width, boundary_points_per_edge)

        flattened_flows = interpolate_spline(
            dest_control_point_locations, control_point_flows,
            flattened_grid_locations, interpolation_order, regularization_weight)

        dense_flows = array_ops.reshape(flattened_flows,
                                        [batch_size, image_height, image_width, 2])

        warped_image = dense_image_warp(image, dense_flows)

        return warped_image, dense_flows

In [13]:
def image_warping(src_img, src_landmarks, dest_landmarks):
    # expanded_src_landmarks = np.expand_dims(np.float32(src_landmarks), axis=0)
    # expanded_dest_landmarks = np.expand_dims(np.float32(dest_landmarks), axis=0)
    # expanded_src_img = np.expand_dims(np.float32(src_img) / 255, axis=0)

    warped_img, dense_flows = sparse_image_warp(src_img,
                          src_landmarks,
                          dest_landmarks,
                          interpolation_order=1,
                          regularization_weight=0.1,
                          num_boundary_points=2,
                          name='sparse_image_warp')

    with tf.Session() as sess:
        out_img = sess.run(warped_img)
        warp_img = np.uint8(out_img[:, :, :, :])
    
    return warp_img
    
def face_landmark(img):
    X = np.zeros((img.shape[0], 72 ,2))
    flag = []
    for i in range(img.shape[0]):

        landmark = face_landmarks(img[i].reshape(224,224,3))
        resultant = []
        try:
            for j in list(landmark[0].keys()):
                resultant += landmark[0][j] 
        except IndexError:
            flag.append(i)
            continue
        X[i] = np.array(resultant)
    return  X, flag

In [14]:
checkpoint_path='/content/drive/My Drive/Speech2Face/models/face_decoder_warping/new_dataset_v5/epoch_26_steps_0.pth'
#checkpoint_path=None
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Set Model
model = DECODER('train').to(device)
optimizer = optim.Adam(model.parameters())
global_epoch = 0

if checkpoint_path:
    checkpoint = torch.load(checkpoint_path)
    global_epoch = checkpoint["epoch"] +1
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

# Set loss and optimizer
criterion = nn.MSELoss()
criterion2 = nn.L1Loss()
alpha = 0.0002
beta  = 1.0




In [15]:
base_dir='/content/drive/My Drive/Speech2Face/'

In [32]:
#import argparse



#from dataloader import EmbedImagePairs
#from face_encoder_model import vgg_face_dag
#from face_decoder_model import Generator




def _save_model(epoch, model, optimizer, steps, fname):
    torch.save({
        'steps': steps,
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, join(output_dir_name, 'epoch_{}_steps_{}.pth'.format(epoch, steps)))


def train_epoch(epoch,model, trn_dataloader, criterion, optimizer, device):
    model.train()

    running_loss = 0
    dataset_len = len(trn_dataloader.dataset)

    count = 0
    numOfElements = 0
    saving_steps = 50

    for x,landmark, y in trn_dataloader:
        
        count += 1
        x = x.squeeze().to(device)  

        # y_img = cv2.cvtColor(np.einsum('abc->bca',y[0].numpy()*255),cv2.COLOR_BGR2RGB)
        # cv2_imshow(y_img)
        y = y.reshape((-1, 3, 224, 224))

        y = y.to(device)
        landmark = landmark.to(device)

        optimizer.zero_grad()

        outLandmark, outputs  = model(x)


        #print("\n\nO", outputs.shape, outputs.dtype) #[-1, 3, 224,244]
        #print("y", y.shape, y.dtype) # [-1, 224,224, 3]


        loss2 = criterion2(outputs, y)
        loss1 = criterion(outLandmark, landmark)

        loss = (alpha*loss1 + beta*loss2 )
        
        loss.backward()
        
        optimizer.step()
        
        # o_img = cv2.cvtColor(np.einsum('abc->bca',outputs[0].cpu().detach().numpy()*255),cv2.COLOR_BGR2RGB)
        #cv2_imshow(o_img)
        running_loss += loss.item() * x.size(0)
        numOfElements += x.shape[0]
        

        
        # outLandmark = outLandmark.cpu().detach().numpy()
        # outLandmark = np.dstack((outLandmark[:,0::2],outLandmark[:,1::2]))
        # print("land np img np ", outL_.shape, img_.shape)
        # img1 = np.einsum('dabc->dbca',outputs.cpu().detach().numpy()*255)
        # img1 = (img1).astype(np.uint8)
        #print("img_t ",img_t.shape, img_t[0])
        # src, flag = face_landmark(img1)
        # if flag:
        #     for r in flag:
        #         src[r] = outLandmark[r]
        # result = image_warping(img1.astype(np.float32), src.astype(np.float32), outLandmark.astype(np.float32))

        # w_img = cv2.cvtColor(result[0], cv2.COLOR_BGR2RGB)
        #cv2_imshow(w_img)

        #print("loss:", running_loss / (x.shape[0] * count) , "Steps:", str(numOfElements) + "/" + str(dataset_len) )

        #if (numOfElements % (x.shape[0] * saving_steps)) == 0:
        #  _save_model(epoch, model, optimizer, numOfElements, output_dir_name)
    
    trn_loss = running_loss/len(trn_dataloader.dataset)
    
    return trn_loss


def train(model, dataloaders, criterion, optimizer, device, output_dir_name, num_epochs=100):
    trn_losses = []

    # make dir where trained model saved
    os.makedirs(output_dir_name, exist_ok=True)

    # start at zero, end at num_epochs (including)
    print("Start Training")
    for epoch in tqdm(range(global_epoch, num_epochs+1)):
        # Training phase
        trn_loss = train_epoch(epoch, model, dataloaders, criterion, optimizer, device)
        trn_losses += [trn_loss]
        print("Epoch: {}  || Loss: {}".format(epoch, trn_loss))
    
        _save_model(epoch, model, optimizer, 0, output_dir_name)


base_dir='/content/drive/My Drive/Speech2Face/'
input="vox1_dataset/unzippedFaces"
batch_size=128
num_epochs=80

# Load face-encoder model
# face_encoder_path = join(base_dir, "Pretrained", "vgg_face_dag.pth")
# face_encoder_model = vgg_face_dag(face_encoder_path).to(device)
# color_mean, color_std = face_encoder_model.meta["mean"], face_encoder_model.meta["std"]
# color_mean = [tmp / 255.0 for tmp in color_mean]
# color_std = [tmp / 255.0 for tmp in color_std]

# Set DataLoader
input_path = '/content/content/Face_Feature/'
print('Image Pairing')
dataset = EmbedImagePairs(input_path)
print('Creating dataloader')
trn_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=8)
output_dir_name = join(base_dir, "models", "face_decoder_warping/new_dataset_v5")

# Train
train(model, trn_dataloader, criterion, optimizer, device, output_dir_name, num_epochs=num_epochs)
print("Finish training")





  0%|          | 0/81 [00:00<?, ?it/s]

Image Pairing
Creating dataloader
Start Training
loss: 52.302608489990234 Steps: 128/12117
loss: 51.53023338317871 Steps: 256/12117
loss: 51.16382853190104 Steps: 384/12117
loss: 51.66936111450195 Steps: 512/12117
loss: 52.076507568359375 Steps: 640/12117
loss: 52.19790776570638 Steps: 768/12117
loss: 52.13435091291155 Steps: 896/12117
loss: 52.18768262863159 Steps: 1024/12117
loss: 52.28740310668945 Steps: 1152/12117
loss: 52.34021987915039 Steps: 1280/12117
loss: 52.34658640081232 Steps: 1408/12117
loss: 52.50662803649902 Steps: 1536/12117
loss: 52.532684326171875 Steps: 1664/12117
loss: 52.58227757045201 Steps: 1792/12117
loss: 52.535779825846355 Steps: 1920/12117
loss: 52.46027421951294 Steps: 2048/12117
loss: 52.39029940436868 Steps: 2176/12117
loss: 52.385089026557075 Steps: 2304/12117
loss: 52.35911700600072 Steps: 2432/12117
loss: 52.31344718933106 Steps: 2560/12117
loss: 52.22888982863653 Steps: 2688/12117
loss: 52.26574568314986 Steps: 2816/12117
loss: 52.25911198491635 Steps





  1%|          | 1/81 [08:04<10:46:26, 484.83s/it]

loss: 50.85444641113281 Steps: 128/12117
loss: 52.554800033569336 Steps: 256/12117
loss: 52.903082529703774 Steps: 384/12117
loss: 52.364752769470215 Steps: 512/12117
loss: 52.39353256225586 Steps: 640/12117
loss: 52.60896301269531 Steps: 768/12117
loss: 52.64117431640625 Steps: 896/12117
loss: 52.8578200340271 Steps: 1024/12117
loss: 52.73579321967231 Steps: 1152/12117
loss: 52.58002281188965 Steps: 1280/12117
loss: 52.53561921553178 Steps: 1408/12117
loss: 52.48404630025228 Steps: 1536/12117
loss: 52.50322048480694 Steps: 1664/12117
loss: 52.61994334629604 Steps: 1792/12117
loss: 52.60984624226888 Steps: 1920/12117
loss: 52.60027360916138 Steps: 2048/12117
loss: 52.57835590138155 Steps: 2176/12117
loss: 52.543329026963974 Steps: 2304/12117
loss: 52.5266103242573 Steps: 2432/12117
loss: 52.49924240112305 Steps: 2560/12117
loss: 52.44857842581613 Steps: 2688/12117
loss: 52.49384897405451 Steps: 2816/12117
loss: 52.40681789232337 Steps: 2944/12117
loss: 52.39076979955038 Steps: 3072/121





  2%|▏         | 2/81 [16:11<10:38:53, 485.24s/it]

...





 32%|███▏      | 26/81 [3:30:48<7:28:12, 488.96s/it]

loss: 51.57698440551758 Steps: 128/12117
loss: 51.147216796875 Steps: 256/12117
loss: 50.885878245035805 Steps: 384/12117
loss: 50.88784885406494 Steps: 512/12117
loss: 50.80407180786133 Steps: 640/12117
loss: 50.64446830749512 Steps: 768/12117
loss: 50.698296138218474 Steps: 896/12117
loss: 50.660314083099365 Steps: 1024/12117
loss: 50.42471610175239 Steps: 1152/12117
loss: 50.454922103881835 Steps: 1280/12117
loss: 50.44344156438654 Steps: 1408/12117
loss: 50.45196374257406 Steps: 1536/12117
loss: 50.35742539625902 Steps: 1664/12117
loss: 50.390184947422576 Steps: 1792/12117
loss: 50.30838241577148 Steps: 1920/12117
loss: 50.267900228500366 Steps: 2048/12117
loss: 50.31824852438534 Steps: 2176/12117
loss: 50.23146438598633 Steps: 2304/12117
loss: 50.19767982081363 Steps: 2432/12117
loss: 50.15151958465576 Steps: 2560/12117
loss: 50.093052092052645 Steps: 2688/12117
loss: 50.18797406283292 Steps: 2816/12117
loss: 50.20161238960598 Steps: 2944/12117
loss: 50.17702563603719 Steps: 3072/





 33%|███▎      | 27/81 [3:38:39<7:15:04, 483.41s/it]

loss: 50.436275482177734 Steps: 128/12117
loss: 51.27895545959473 Steps: 256/12117
loss: 50.72018814086914 Steps: 384/12117
loss: 50.452176094055176 Steps: 512/12117
loss: 50.02598648071289 Steps: 640/12117
loss: 50.05006090799967 Steps: 768/12117
loss: 49.981743403843474 Steps: 896/12117
loss: 49.84634351730347 Steps: 1024/12117
loss: 49.87387212117513 Steps: 1152/12117
loss: 49.974727630615234 Steps: 1280/12117
loss: 49.95232079245827 Steps: 1408/12117
loss: 49.891844749450684 Steps: 1536/12117
loss: 49.930841592641976 Steps: 1664/12117
loss: 50.09318733215332 Steps: 1792/12117
loss: 50.18232905069987 Steps: 1920/12117
loss: 50.23295569419861 Steps: 2048/12117
loss: 50.28368018655216 Steps: 2176/12117
loss: 50.32345305548774 Steps: 2304/12117
loss: 50.25975638941715 Steps: 2432/12117
loss: 50.26215591430664 Steps: 2560/12117
loss: 50.218303862072176 Steps: 2688/12117
loss: 50.20534844831987 Steps: 2816/12117
loss: 50.180758600649625 Steps: 2944/12117
loss: 50.08353519439697 Steps: 30

IndexError: ignored

In [ ]:
base_dir='/content/drive/My Drive/Speech2Face/'
face_encoder_path = join(base_dir, "Pretrained", "vgg_face_dag.pth")
face_encoder_model = vgg_face_dag(face_encoder_path)
face_encoder_model.eval()
face_encoder_model.to(device)

color_mean, color_std = face_encoder_model.meta["mean"], face_encoder_model.meta["std"]
color_mean = [tmp / 255.0 for tmp in color_mean]
color_std = [tmp / 255.0 for tmp in color_std]
model.eval()
#img = Image.open('/content/338x338-10-graziano-frontal.png')
#img = Image.open('/content/1966c850f1620718bc846793a0a0d78f.jpg')
img = Image.open('jolie.jpg')
transform_fe = transforms.Compose([
            transforms.Resize((224, 224)),
            _Normalize_Tensor(color_mean, color_std)
        ])


t_img = transform_fe(img).unsqueeze(0).to(device)
emb = face_encoder_model(t_img)
land_out, outputs  = model(emb)
display(img)


In [ ]:
o_img = cv2.cvtColor(np.einsum('abc->bca',outputs[0].cpu().detach().numpy()*255),cv2.COLOR_BGR2RGB)
cv2_imshow(o_img)

outLandmark = land_out.cpu().detach().numpy()
# outLandmark = np.dstack((outLandmark[:,0::2],outLandmark[:,1::2]))
print(img.size)
print(t_img.shape)

outLandmark = outLandmark.reshape((-1, 72, 2))
#outLandmark = np.interp(outLandmark, (outLandmark.min(), outLandmark.max()), (0, 224))


# print("land np img np ", outL_.shape, img_.shape)
img1 = np.einsum('dabc->dbca',outputs.cpu().detach().numpy()*255)
img1 = (img1).astype(np.uint8)
# print("img_t ",img_t.shape, img_t[0])
src, flag = face_landmark(img1)
if flag:
    for r in flag:
        src[r] = outLandmark[r] 

diff = np.median(outLandmark, axis=1) - np.median(src, axis=1)

right_list = set()
for i in range(outLandmark.shape[1]):
  if not (outLandmark[0,i,0] == 0 or outLandmark[0,i,1] == 0) :
    right_list.add(i)

good_list = np.array(list(right_list))
src_new = src[:,good_list]
outLandmark_new = outLandmark[:,good_list]
#outLandmark_new[:,:] -= diff
result = image_warping(img1.astype(np.float32), src_new.astype(np.float32), outLandmark_new.astype(np.float32))
w_img = cv2.cvtColor(result[0], cv2.COLOR_BGR2RGB)
cv2_imshow(w_img)

In [ ]:
import matplotlib.pyplot as plt

def plotLandmarks(img, landmarks):
  img_p = img.squeeze(0)
  fig = plt.figure(figsize=(15, 5))
  ax = fig.add_subplot(1, 3, 1)
  ax.imshow(img_p)
  ax = fig.add_subplot(1, 3, 2)
  ax.scatter(landmarks[:,0], -landmarks[:,1], alpha=0.8)
  ax = fig.add_subplot(1, 3, 3)
  img2 = img_p.copy()

  for p in landmarks[:].astype(np.uint8):
      img2[p[1]-2:p[1]+2, p[0]-2:p[0]+2, :] = (255, 255, 255)
      # note that the values -3 and +3 will make the landmarks
      # overlayed on the image 6 pixels wide; depending on the
      # resolution of the face image, you may want to change
      # this value


  ax.imshow(img2)
  plt.show()

plotLandmarks(img1, src.squeeze())


In [ ]:
im = Image.fromarray(img1[0])
im = im.resize((100,100))

img1 = np.array(im).reshape((-1, 100, 100, 3))
plotLandmarks(img1, outLandmark_new[0])

In [ ]:
im = Image.fromarray(np.uint8(o_img))
b, g, r = im.split()
im = Image.merge("RGB", (r, g, b))

im.save("/content/test/jolie.png")

In [ ]:
!cp donna.png /content/test

In [ ]:
!git clone https://github.com/hieubkset/Pytorch-Image-Deblurring.git

In [ ]:
!pip install -r /content/Pytorch-Image-Deblurring/requirements.txt

In [ ]:
!python /content/Pytorch-Image-Deblurring/demo.py --gpu 0 --train_dir /content/Pytorch-Image-Deblurring/pretrained --exp_name multi_skip --image /content/test/jolie.png